In [ ]:
from sage.graphs.ribbon_graph import RibbonGraph

# Orden cíclico alrededor del vértice: 1 → 2 → 3 → 4 → 1
sigma = '(1,2,3,4)'

# Emparejamiento de semiaristas en aristas:
# - Arista 1: semiaristas 1 y 2
# - Arista 2: semiaristas 3 y 4
rho = '(1,2)(3,4)'

# Crear el ribbon graph
R = RibbonGraph(sigma, rho)

# Calcular invariantes
print(f"Número de vértices: {R.num_vertices()}")
print(f"Número de aristas: {R.num_edges()}")
print(f"Género: {R.genus()}")
print(f"Componentes de frontera: {R.number_boundaries()}")

Verificando entorno de SageMath...
Versión de SageMath: SageMath version 10.7, Release Date: 2025-08-09
Versión de Python: 3.13.3 (main, May 16 2025, 18:02:35) [Clang 17.0.0 (clang-1700.0.13.3)]
✓ RibbonGraph disponible
✓ PermutationGroupElement disponible
✓ Funciones 3D (polygon3d, line3d) disponibles
✓ sphere() disponible globalmente
✓ NumPy disponible (versión 2.2.4)

Entorno verificado correctamente. Listo para trabajar!


# 📘 Ribbon Graphs - Sistema de Visualización

Este notebook te permite trabajar con **ribbon graphs** (grafos de cinta) usando SageMath.

## ⚡ Inicio Rápido

**IMPORTANTE**: Ejecuta las celdas en orden, de arriba hacia abajo.

1. **Verificación del entorno** ⬇️ (celda siguiente)
2. **Definición de clases** (clase `RibbonGraphVisualizer`)
3. **Ejemplos y visualizaciones**

---

In [ ]:
import numpy as np
from sage.geometry.ribbon_graph import RibbonGraph
from sage.groups.perm_gps.permgroup import SymmetricGroup
from sage.plot.plot3d.shapes2 import polygon3d, line3d

class RibbonGraphVisualizer:
    """
    Clase para visualización y análisis de Ribbon Graphs.
    Combina la potencia combinatoria de SageMath con visualizaciones 2D/3D.
    """
    
    def __init__(self, sigma, rho, n=None):
        """
        Inicializa un ribbon graph.
        
        Args:
            sigma: Permutación de vértices (ciclos alrededor de vértices)
                   Puede ser una cadena como '(1,2,3)(4,5,6)' o un elemento del grupo simétrico
            rho: Permutación de aristas (involución que empareja dardos)
                 Puede ser una cadena como '(1,4)(2,5)(3,6)' o un elemento del grupo simétrico
            n: Tamaño del grupo simétrico (opcional, se deduce si no se proporciona)
        """
        # Determinar el tamaño del grupo si es necesario
        if isinstance(sigma, str) or isinstance(rho, str):
            # Extraer el número máximo de los ciclos
            import re
            nums_sigma = [int(x) for x in re.findall(r'\d+', sigma if isinstance(sigma, str) else str(sigma))]
            nums_rho = [int(x) for x in re.findall(r'\d+', rho if isinstance(rho, str) else str(rho))]
            if n is None:
                n = max(nums_sigma + nums_rho)
        
        # Crear grupo simétrico
        self.S = SymmetricGroup(n)
        
        # Convertir strings a elementos del grupo simétrico
        if isinstance(sigma, str):
            sigma = self.S(sigma)
        else:
            sigma = self.S(sigma)
            
        if isinstance(rho, str):
            rho = self.S(rho)
        else:
            rho = self.S(rho)
            
        self.sigma = sigma
        self.rho = rho
        self.ribbon_graph = RibbonGraph(sigma, rho)
        self.graph = self.ribbon_graph.graph()
        
    def invariantes(self):
        """Calcula invariantes topológicos del ribbon graph."""
        return {
            'genus': self.ribbon_graph.genus(),
            'caras': self.ribbon_graph.number_of_faces(),
            'vertices': self.graph.num_verts(),
            'aristas': self.graph.num_edges(),
            'componentes_frontera': self.ribbon_graph.number_of_boundaries(),
            'euler_char': self.ribbon_graph.euler_characteristic()
        }
    
    def mostrar_invariantes(self):
        """Imprime los invariantes de forma legible."""
        inv = self.invariantes()
        print("="*50)
        print("INVARIANTES DEL RIBBON GRAPH")
        print("="*50)
        print(f"Género (g):                    {inv['genus']}")
        print(f"Vértices (V):                  {inv['vertices']}")
        print(f"Aristas (E):                   {inv['aristas']}")
        print(f"Caras (F):                     {inv['caras']}")
        print(f"Componentes de frontera (b):  {inv['componentes_frontera']}")
        print(f"Característica de Euler (χ):  {inv['euler_char']}")
        print("="*50)
        
    def visualizar_2d(self, layout='spring', vertex_size=300, edge_width=2, 
                     mostrar_dardos=True, mostrar_labels=True):
        """
        Visualización 2D del ribbon graph.
        
        Args:
            layout: Tipo de layout ('spring', 'circular', 'planar')
            vertex_size: Tamaño de los vértices
            edge_width: Grosor de las aristas
            mostrar_dardos: Si mostrar orientación de dardos
            mostrar_labels: Si mostrar etiquetas
        """
        G = self.graph
        
        # Calcular posiciones
        if layout == 'circular':
            pos = G.layout_circular()
        elif layout == 'planar':
            try:
                pos = G.layout_planar()
            except:
                print("El grafo no es planar, usando spring layout")
                pos = G.layout_spring()
        else:
            pos = G.layout_spring()
        
        # Crear visualización usando el método de plot de SageMath
        return G.plot(pos=pos, vertex_size=vertex_size, edge_thickness=edge_width,
                     vertex_labels=mostrar_labels)
    
    def visualizar_3d_cintas(self, layout='spring', iterations=500, 
                            ancho_cinta=0.15, opacity=0.7):
        """
        Visualización 3D que muestra las 'cintas' del ribbon graph.
        
        Args:
            layout: Tipo de layout
            iterations: Iteraciones para el layout spring
            ancho_cinta: Ancho de las cintas visualizadas
            opacity: Opacidad de las cintas
        """
        G = self.graph
        
        # Layout 3D
        pos = G.layout(layout=layout, dim=3, iterations=iterations, seed=42)
        
        # Inicializar escena vacía
        escena = None
        
        # Dibujar Vértices como esferas
        # sphere() está disponible globalmente en SageMath
        for v in G.vertices():
            esfera = sphere(center=pos[v], size=0.15, color='red')
            if escena is None:
                escena = esfera
            else:
                escena += esfera
            
        # Dibujar Cintas
        visited = set()
        
        for u, v, _ in G.edges():
            # Evitar duplicar aristas
            key = tuple(sorted([str(u), str(v)]))
            if key in visited:
                continue
            visited.add(key)
            
            p1 = np.array(pos[u])
            p2 = np.array(pos[v])
            vec = p2 - p1
            
            # Vector perpendicular para el ancho de la cinta
            mid = (p1 + p2) / 2
            if np.linalg.norm(mid) > 1e-5:
                up = mid / np.linalg.norm(mid)
            else:
                up = np.array([0, 0, 1])
            
            perp = np.cross(vec, up)
            if np.linalg.norm(perp) > 1e-5:
                perp = perp / np.linalg.norm(perp) * ancho_cinta
            else:
                perp = np.array([ancho_cinta, 0, 0])
            
            # Polígono de la cinta
            pts = [
                tuple(p1 + perp),
                tuple(p2 + perp),
                tuple(p2 - perp),
                tuple(p1 - perp)
            ]
            
            escena += polygon3d(pts, color='cyan', opacity=opacity)
            escena += line3d([tuple(p1), tuple(p2)], color='black', thickness=3)
        
        return escena
    
    def exportar_info(self):
        """Exporta información completa del ribbon graph."""
        inv = self.invariantes()
        info = {
            'invariantes': inv,
            'sigma': str(self.sigma),
            'rho': str(self.rho),
            'vertices': list(self.graph.vertices()),
            'aristas': list(self.graph.edges()),
        }
        return info

# =============================================================================
# EJEMPLO 1: Ribbon Graph simple
# =============================================================================
print("Ejemplo 1: Ribbon Graph simple")

# Crear las permutaciones correctamente usando el grupo simétrico
sigma1 = '(1,2,3)(4,5,6)'
rho1 = '(1,4)(2,5)(3,6)'

viz1 = RibbonGraphVisualizer(sigma1, rho1)
viz1.mostrar_invariantes()

# Visualización 2D
print("\nGenerando visualización 2D...")
plot_2d = viz1.visualizar_2d(layout='spring', mostrar_labels=True)
plot_2d.show(figsize=8)